In [2]:
!pip install swig


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install "gymnasium[box2d]"

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.6 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/10.6 MB 5.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/10.6 MB 5.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/10.6 MB 5.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/10.6 MB 1.5 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/10.6 MB 1.4 MB/s eta 0:00:07
   -------- ------------------------------- 2.4/10.6 MB 1.6 MB/s eta 0:00:06
   ----------- ---------------------------- 3.1/10.6 MB 1.9 MB/s eta 0:00:04
   --------------- ------------------------ 4.2/10.6 MB 2.4 MB/s eta 0:00:03
   ------------------ --------------------- 5.0/10.6 MB 2.5 MB/s eta 0:00:03
   ---------------------- ----------------- 6.0/10.6 MB 2.


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import gymnasium as gym
import math
import torch.nn.functional as F

In [5]:
# Класс нейрона скрытого слоя 
# === Класс нейрона скрытого слоя ===
class Neuron:
    def __init__(self, input_size):
        self.weights = np.random.randn(input_size) * math.sqrt(2. / input_size)
        self.bias = 0.0

    def activate(self, x):
        z = np.dot(x, self.weights) + self.bias
        return np.tanh(z)
n = Neuron(input_size=8)
print(n.weights)
x = np.random.randn(8)
output = n.activate(x)
print("Neuron output:", output)

[-0.13703014  0.74272816  0.33811851  0.04608804  0.10973777  0.35198605
  0.02301245 -0.23384887]
Neuron output: 0.6080787177621686


In [6]:
# Популяция нейронов скрытого слоя 
class NeuronPopulation:
    def __init__(self, input_size, population_size):
        self.neurons = [Neuron(input_size) for _ in range(population_size)]

    def mutate(self, neuron, mutation_strength):
        new_neuron = Neuron(len(neuron.weights))
        new_neuron.weights = neuron.weights + np.random.normal(0, mutation_strength, size=neuron.weights.shape)
        new_neuron.bias = neuron.bias + np.random.normal(0, mutation_strength)
        return new_neuron

    def evolve(self, fitnesses, mutation_strength):
        sorted_indices = np.argsort(fitnesses)[::-1]
        top = [self.neurons[i] for i in sorted_indices[:len(self.neurons)//2]]
        new_population = []
        for neuron in top:
            new_population.append(neuron)
            new_population.append(self.mutate(neuron, mutation_strength))
        self.neurons = new_population[:len(self.neurons)]

pop = NeuronPopulation(input_size=8, population_size=5)
for i in range(5):
  print(pop.neurons[i])
print("Веса первого нейрона:", pop.neurons[0].weights)

mut_neuron = pop.mutate(pop.neurons[0], mutation_strength=0.1)
print("Мутировавшие веса:", mut_neuron.weights)

fitnesses = np.random.randn(5)
pop.evolve(fitnesses, mutation_strength=0.1)
print("Поплуяция эволюционировала - новые веса первого нейрона:", pop.neurons[0].weights)


Веса первого нейрона: [ 0.17665049  0.98453998 -0.54107708 -0.19167655  0.38970937  0.18731136
 -0.75703331 -0.22389388]
Мутировавшие веса: [ 0.21147102  0.93296466 -0.48894403 -0.35066851  0.29409809  0.2661975
 -0.73207824 -0.17109773]
Поплуяция эволюционировала - новые веса первого нейрона: [ 0.33991579 -0.15939845  0.17774244  0.45318757  0.17383878 -0.26713338
 -1.04978021 -0.60431183]


In [9]:
class OutputLayer:
    def __init__(self, hidden_size, output_size):
        self.weights = np.random.randn(hidden_size, output_size)

    def forward(self, hidden_output):
        return np.dot(hidden_output, self.weights)
    
ol = OutputLayer(hidden_size=4, output_size=3)
hidden_output = np.array([0.5, -0.2, 0.1, 0.9])
output = ol.forward(hidden_output)
print("Выход:", output)

Выход: [-1.35493425  0.0409108   0.01991938]
